In [1]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName("sanity")
         .master("local[*]")
         .getOrCreate())
spark.range(10).selectExpr("sum(id) as total").show()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/07 00:37:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-----+
|total|
+-----+
|   45|
+-----+



In [2]:
from datasets import load_dataset, get_dataset_config_names

print(get_dataset_config_names("eloukas/edgar-corpus"))

ds = load_dataset("eloukas/edgar-corpus", "full", split="train")

/Users/anant/anaconda3/envs/spark/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['full', 'year_1993', 'year_1994', 'year_1995', 'year_1996', 'year_1997', 'year_1998', 'year_1999', 'year_2000', 'year_2001', 'year_2002', 'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 'year_2018', 'year_2019', 'year_2020']


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from datasets import load_dataset

# 1. Start Spark
spark = SparkSession.builder.appName("edgar-aig-sec10").getOrCreate()

# 2. Convert only required columns
pdf = ds.select_columns(["filename", "cik", "year", "section_10"]).to_pandas()
sdf = spark.createDataFrame(pdf)


25/09/07 00:37:25 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
# 3. Filter by AIG CIK
AIG_CIK = "0000005272"
aig_sec10_df = sdf.filter(col("cik") == AIG_CIK)

# 4. Inspect results
aig_sec10_df.select("filename", "year", "section_10").show(5, truncate=False)

25/09/07 00:37:36 WARN TaskSetManager: Stage 3 contains a task of very large size (49783 KiB). The maximum recommended task size is 1000 KiB.
25/09/07 00:37:37 WARN TaskSetManager: Stage 4 contains a task of very large size (54747 KiB). The maximum recommended task size is 1000 KiB.
25/09/07 00:37:37 ERROR Inbox: An error happened while processing message in the inbox for LocalSchedulerBackendEndpoint
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.util.Arrays.copyOf(Arrays.java:3537)
	at java.base/java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:100)
	at java.base/java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:130)
	at org.apache.spark.util.ByteBufferOutputStream.write(ByteBufferOutputStream.scala:41)
	at java.base/java.io.ObjectOutputStream$BlockDataOutputStream.drain(ObjectOutputStream.java:1886)
	at java.base/java.io.ObjectOutputStream$BlockDataOutputStream.setBlockDataMode(ObjectOutputStream.java:1795)
	at java.base/java.io.Obj